In [1]:
%pip install -r requirements.txt

import pandas as pd
import numpy as np

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.2.2 -> 22.3
[notice] To update, run: C:\Users\Admin\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip


In [148]:
#interest rate

df = pd.read_csv('./data/before-convert/interest_rate.csv')
df['year'] = pd.DatetimeIndex(df['TIME']).year
df['month'] = pd.DatetimeIndex(df['TIME']).month

df = df.drop('TIME', axis=1)

df = df.rename(columns = {'LOCATION':'location'})
df = df.drop('Country', axis=1)

longTerm = df.query('Subject == "Long-term interest rates, Per cent per annum"')
longTerm = longTerm.rename(columns = {'Interest-Rate':'interest_rate_long_term'})
longTerm = longTerm.drop('Subject', axis=1)

shortTerm = df.query('Subject == "Short-term interest rates, Per cent per annum"')
shortTerm = shortTerm.rename(columns = {'Interest-Rate':'interest_rate_short_term'})
shortTerm = shortTerm.drop('Subject', axis=1)


immediateTerm = df.query('Subject == "Immediate interest rates, Call Money, Interbank Rate, Per cent per annum"')
immediateTerm = immediateTerm.rename(columns = {'Interest-Rate':'interest_rate_immediate_term'})
immediateTerm = immediateTerm.drop('Subject', axis=1)

df = pd.merge(longTerm, shortTerm, on=['location', 'year', 'month'] , how='inner', suffixes=('', '_delme'))
df = pd.merge(df, immediateTerm, on=['location', 'year', 'month'] , how='inner', suffixes=('', '_delme'))
df = df[[c for c in df.columns if not c.endswith('_delme')]]

df.to_csv('./data/interest_rate.csv', index=False)

In [229]:
#cpi

#au
df_au = pd.read_csv('./data/before-convert/cpi_au.csv')

df_au['Date'] = pd.DatetimeIndex(df_au['TIME'])
df_au = df_au.drop('TIME', axis=1)

df_au['Date'] = pd.to_datetime(df_au['Date']).dt.to_period('M')
df_au = df_au.set_index('Date').resample('M').interpolate()
df_au['location'] = 'AUS'
df_au['year'] = pd.DatetimeIndex(df_au.index.to_timestamp()).year
df_au['month'] = pd.DatetimeIndex(df_au.index.to_timestamp()).month

df_au.reset_index(drop=True, inplace=True)


#us
df_us = pd.read_csv('./data/before-convert/cpi_us.csv')

df_final_us = pd.DataFrame(columns=['year','month'])

for index, row in df_us.iterrows():
    for i in range(12):
        df_final_us = pd.concat([df_final_us, pd.DataFrame.from_records([{'year':row[0],'month':i+1, 'cpi-index':row[i+1]}])])

df_final_us['location'] = 'USA'
df_final_us = df_final_us.dropna()
df_final_us['year'] = df_final_us['year'].astype(int)
    
df = pd.concat([df_au, df_final_us])

df.to_csv('./data/cpi.csv', index=False)

In [297]:
#energy_price

#au

df_au = pd.read_csv('./data/before-convert/energy_price_au.csv')

df_au['year'] = pd.DatetimeIndex(df_au['Date'], dayfirst=True).year
df_au['month'] = pd.DatetimeIndex(df_au['Date'], dayfirst=True).month

df_au = df_au.rename(columns = {'Value':'energy_price'})


df_au['Date'] = pd.to_datetime(df_au['Date'],dayfirst=True)
df_au = df_au.set_index('Date').resample('M').median(numeric_only=True)

df_au.head(100)

df_au['energy_price_lag_1m'] = df_au['energy_price'].shift(freq='1M')
df_au['energy_price_lag_2m'] = df_au['energy_price'].shift(freq='2M')
df_au['energy_price_lag_3m'] = df_au['energy_price'].shift(freq='3M')
df_au['energy_price_lag_4m'] = df_au['energy_price'].shift(freq='4M')
df_au['energy_price_lag_5m'] = df_au['energy_price'].shift(freq='5M')
df_au['energy_price_lag_6m'] = df_au['energy_price'].shift(freq='6M')
df_au['energy_price_lag_7m'] = df_au['energy_price'].shift(freq='7M')
df_au['energy_price_lag_8m'] = df_au['energy_price'].shift(freq='8M')
df_au['energy_price_lag_9m'] = df_au['energy_price'].shift(freq='9M')
df_au['energy_price_lag_10m'] = df_au['energy_price'].shift(freq='10M')
df_au['energy_price_lag_11m'] = df_au['energy_price'].shift(freq='11M')
df_au['energy_price_lag_12m'] = df_au['energy_price'].shift(freq='12M')

df_au.reset_index(drop=True, inplace=True)

df_au['year'] = df_au['year'].astype(int)
df_au['month'] = df_au['month'].astype(int)
df_au['location'] = 'AUS'

#us

df_us = pd.read_csv('./data/before-convert/energy_price_us.csv')

df_us['year'] = pd.DatetimeIndex(df_us['date'], dayfirst=True).year
df_us['month'] = pd.DatetimeIndex(df_us['date'], dayfirst=True).month
df_us['location'] = 'USA'
df_us = df_us.rename(columns = {'value':'energy_price'})
df_us = df_us.set_index('date')

df_us['energy_price_lag_1m'] = df_us['energy_price'].shift(1)
df_us['energy_price_lag_2m'] = df_us['energy_price'].shift(2)
df_us['energy_price_lag_3m'] = df_us['energy_price'].shift(3)
df_us['energy_price_lag_4m'] = df_us['energy_price'].shift(4)
df_us['energy_price_lag_5m'] = df_us['energy_price'].shift(5)
df_us['energy_price_lag_6m'] = df_us['energy_price'].shift(6)
df_us['energy_price_lag_7m'] = df_us['energy_price'].shift(7)
df_us['energy_price_lag_8m'] = df_us['energy_price'].shift(8)
df_us['energy_price_lag_9m'] = df_us['energy_price'].shift(9)
df_us['energy_price_lag_10m'] = df_us['energy_price'].shift(10)
df_us['energy_price_lag_11m'] = df_us['energy_price'].shift(11)
df_us['energy_price_lag_12m'] = df_us['energy_price'].shift(12)

df_us.reset_index(drop=True, inplace=True)

df_us.head(100)

df = pd.concat([df_au, df_us])


df = df.dropna()

df.to_csv('./data/energy_price.csv', index=False)

In [295]:
#au

df_au = pd.read_csv('./data/before-convert/energy_price_au.csv')

df_au['year'] = pd.DatetimeIndex(df_au['Date'], dayfirst=True).year
df_au['month'] = pd.DatetimeIndex(df_au['Date'], dayfirst=True).month

df_au = df_au.rename(columns = {'Value':'energy_price'})


df_au['Date'] = pd.to_datetime(df_au['Date'],dayfirst=True)
df_au = df_au.set_index('Date').resample('M').median(numeric_only=True)

df_au.head(100)

df_au['energy_price_lag_1m'] = df_au['energy_price'].shift(freq='1M')
df_au['energy_price_lag_2m'] = df_au['energy_price'].shift(freq='2M')
df_au['energy_price_lag_3m'] = df_au['energy_price'].shift(freq='3M')
df_au['energy_price_lag_4m'] = df_au['energy_price'].shift(freq='4M')
df_au['energy_price_lag_5m'] = df_au['energy_price'].shift(freq='5M')
df_au['energy_price_lag_6m'] = df_au['energy_price'].shift(freq='6M')
df_au['energy_price_lag_7m'] = df_au['energy_price'].shift(freq='7M')
df_au['energy_price_lag_8m'] = df_au['energy_price'].shift(freq='8M')
df_au['energy_price_lag_9m'] = df_au['energy_price'].shift(freq='9M')
df_au['energy_price_lag_10m'] = df_au['energy_price'].shift(freq='10M')
df_au['energy_price_lag_11m'] = df_au['energy_price'].shift(freq='11M')
df_au['energy_price_lag_12m'] = df_au['energy_price'].shift(freq='12M')

df_au.reset_index(drop=True, inplace=True)

df_au['year'] = df_au['year'].astype(int)
df_au['month'] = df_au['month'].astype(int)
df_au['location'] = 'AUS'

#us

df_us = pd.read_csv('./data/before-convert/energy_price_us.csv')

df_us['year'] = pd.DatetimeIndex(df_us['date'], dayfirst=True).year
df_us['month'] = pd.DatetimeIndex(df_us['date'], dayfirst=True).month
df_us['location'] = 'USA'
df_us = df_us.rename(columns = {'value':'energy_price'})
df_us = df_us.set_index('date')

df_us['energy_price_lag_1m'] = df_us['energy_price'].shift(1)
df_us['energy_price_lag_2m'] = df_us['energy_price'].shift(2)
df_us['energy_price_lag_3m'] = df_us['energy_price'].shift(3)
df_us['energy_price_lag_4m'] = df_us['energy_price'].shift(4)
df_us['energy_price_lag_5m'] = df_us['energy_price'].shift(5)
df_us['energy_price_lag_6m'] = df_us['energy_price'].shift(6)
df_us['energy_price_lag_7m'] = df_us['energy_price'].shift(7)
df_us['energy_price_lag_8m'] = df_us['energy_price'].shift(8)
df_us['energy_price_lag_9m'] = df_us['energy_price'].shift(9)
df_us['energy_price_lag_10m'] = df_us['energy_price'].shift(10)
df_us['energy_price_lag_11m'] = df_us['energy_price'].shift(11)
df_us['energy_price_lag_12m'] = df_us['energy_price'].shift(12)

df_us.reset_index(drop=True, inplace=True)

df_us.head(100)

df = pd.concat([df_au, df_us])

df.dropna()

,energy_price,year,month,energy_price_lag_1m,energy_price_lag_2m,energy_price_lag_3m,energy_price_lag_4m,energy_price_lag_5m,energy_price_lag_6m,energy_price_lag_7m,energy_price_lag_8m,energy_price_lag_9m,energy_price_lag_10m,energy_price_lag_11m,energy_price_lag_12m,location
12,30.37,2004,1,29.64,28.515,28.160,26.360,28.800,27.380,26.70,26.17,25.38,29.80,31.845,30.430,AUS
13,29.73,2004,2,30.37,29.640,28.515,28.160,26.360,28.800,27.38,26.70,26.17,25.38,29.800,31.845,AUS
14,32.30,2004,3,29.73,30.370,29.640,28.515,28.160,26.360,28.80,27.38,26.70,26.17,25.380,29.800,AUS
15,32.54,2004,4,32.30,29.730,30.370,29.640,28.515,28.160,26.36,28.80,27.38,26.70,26.170,25.380,AUS
16,36.40,2004,5,32.54,32.300,29.730,30.370,29.640,28.515,28.16,26.36,28.80,27.38,26.700,26.170,AUS
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
388,109.55,2022,5,101.78,108.500,91.640,83.220,71.710,79.150,81.48,71.65,67.73,72.49,71.380,65.170,USA
389,114.84,2022,6,109.55,101.780,108.500,91.640,83.220,71.710,79.15,81.48,71.65,67.73,72.490,71.380,USA
390,101.62,2022,7,114.84,109.550,101.780,108.500,91.640,83.220,71.71,79.15,81.48,71.65,67.730,72.490,USA
391,93.67,2022,8,101.62,114.840,109.550,101.780,108.500,91.640,83.22,71.71,79.15,81.48,71.650,67.730,USA
